In [1]:
!pip install lime
!pip install shap

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
import os
from sklearn.svm import SVC
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_iris
import lime
import lime.lime_tabular
import shap
import psutil
import time
import os

In [3]:
data=pd.read_csv("KDDTrain+.txt")

In [4]:
data.head()

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,0.17,0.03,0.17.1,0.00.6,0.00.7,0.00.8,0.05,0.00.9,normal,20
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125972 entries, 0 to 125971
Data columns (total 43 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   0         125972 non-null  int64  
 1   tcp       125972 non-null  object 
 2   ftp_data  125972 non-null  object 
 3   SF        125972 non-null  object 
 4   491       125972 non-null  int64  
 5   0.1       125972 non-null  int64  
 6   0.2       125972 non-null  int64  
 7   0.3       125972 non-null  int64  
 8   0.4       125972 non-null  int64  
 9   0.5       125972 non-null  int64  
 10  0.6       125972 non-null  int64  
 11  0.7       125972 non-null  int64  
 12  0.8       125972 non-null  int64  
 13  0.9       125972 non-null  int64  
 14  0.10      125972 non-null  int64  
 15  0.11      125972 non-null  int64  
 16  0.12      125972 non-null  int64  
 17  0.13      125972 non-null  int64  
 18  0.14      125972 non-null  int64  
 19  0.15      125972 non-null  int64  
 20  0.16

In [6]:
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted'
,'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate'
,'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','level'])


In [7]:
data.columns = columns
data = data.drop('level', axis=1)
data.tail()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
125967,0,tcp,private,S0,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.0,1.00,1.0,0.00,0.0,neptune
125968,8,udp,private,SF,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.0,0.00,0.0,0.00,0.0,normal
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.0,0.72,0.0,0.01,0.0,normal
125970,0,tcp,klogin,S0,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.0,1.00,1.0,0.00,0.0,neptune
125971,0,tcp,ftp_data,SF,151,0,0,0,0,0,...,77,0.30,0.03,0.30,0.0,0.00,0.0,0.00,0.0,normal


In [8]:
# changing attack labels to their respective attack class
def change_label(df):
  df.attack.replace(['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm'],'Dos',inplace=True)
  df.attack.replace(['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail','snmpgetattack','snmpguess','spy','warezclient','warezmaster','xlock','xsnoop'],'R2L',inplace=True)
  df.attack.replace(['ipsweep','mscan','nmap','portsweep','saint','satan'],'Probe',inplace=True)
  df.attack.replace(['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm'],'U2R',inplace=True)

change_label(data)

le = LabelEncoder()
data['protocol_type']=le.fit_transform(data['protocol_type'])
data['service']=le.fit_transform(data['service'])
data['flag']=le.fit_transform(data['flag'])
data['attack']=le.fit_transform(data['attack'])

def cross_val(x_train, y_train, model):
    accuracies = cross_val_score(estimator = model, X = x_train, y = y_train, cv=5)
    return accuracies.mean()

def fit_and_evaluate(model, x_train , x_test , y_train , y_test):
    model.fit(x_train, y_train)

    model_pred = model.predict(x_test)
    model_cross = cross_val(x_train, y_train, model)

    return model_cross

def run_experiment(model, x_train , x_test , y_train , y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("R^2 : ", r2_score(y_test, y_pred))
    print("MAE :", mean_absolute_error(y_test,y_pred))
    print("RMSE:",np.sqrt(mean_squared_error(y_test, y_pred)))
    report=classification_report(y_test,y_pred)
    print(report)

x = data.drop(['attack'], axis=1).values
y = data['attack'].values


In [9]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
import tracemalloc
import time
data = pd.read_csv('KDDTrain+.txt')
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted'
,'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate'
,'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','level'])
data.columns = columns
data = data.drop('level', axis=1)
data.tail()
le = LabelEncoder()
data['protocol_type'] = le.fit_transform(data['protocol_type'])
data['service'] = le.fit_transform(data['service'])
data['flag'] = le.fit_transform(data['flag'])
data['attack'] = le.fit_transform(data['attack'])
x = data.drop(['attack'], axis=1)
y = data['attack']
tracemalloc.start()
start_time = time.time()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(" RandomForestClassifier Accuracy:", accuracy)
current, peak = tracemalloc.get_traced_memory()
print(f"memory usage: {peak / 10**6} MB")
end_time =time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.4f} seconds")
tracemalloc.stop()

 RandomForestClassifier Accuracy: 0.9984520738241714
memory usage: 121.202037 MB
Elapsed Time: 18.3759 seconds


In [10]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
import tracemalloc
import time
data = pd.read_csv('KDDTrain+.txt')
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted'
,'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate'
,'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','level'])
data.columns = columns
data = data.drop('level', axis=1)
data.tail()
le = LabelEncoder()
data['protocol_type'] = le.fit_transform(data['protocol_type'])
data['service'] = le.fit_transform(data['service'])
data['flag'] = le.fit_transform(data['flag'])
data['attack'] = le.fit_transform(data['attack'])
x = data.drop(['attack'], axis=1)
y = data['attack']
tracemalloc.start()
start_time = time.time()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=50)
clf = RandomForestClassifier(n_estimators=50, criterion='entropy', max_depth=None, min_samples_split=4, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                              max_features='log2', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False,
                              n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("RandomForestClassifier Accuracy:", accuracy)
current, peak = tracemalloc.get_traced_memory()
print(f"memory usage: {peak / 10**6} MB")
end_time =time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.4f} seconds")
tracemalloc.stop()

RandomForestClassifier Accuracy: 0.9980948348856901
memory usage: 114.959061 MB
Elapsed Time: 12.5380 seconds


MLPclassifier

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

mlp = MLPClassifier(hidden_layer_sizes=(150, 100, 50), max_iter=10, alpha=1e-5,
                    solver='adam', verbose=10, tol=1e-4, random_state=42,
                    learning_rate_init=0.001, batch_size=64, early_stopping=True, validation_fraction=0.1)

tracemalloc.start()

start_time = time.time()

mlp.fit(x_train_scaled, y_train)
y_pred_mlp = mlp.predict(x_test_scaled)

accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print("MLPClassifier Accuracy: ", accuracy_mlp)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

end_time = time.time()
elapsed_time = end_time - start_time

tracemalloc.stop()

print(f"Elapsed Time: {elapsed_time:.2f} seconds")


Iteration 1, loss = 0.11974618
Validation score: 0.989453
Iteration 2, loss = 0.03499875
Validation score: 0.992062
Iteration 3, loss = 0.03011508
Validation score: 0.990587
Iteration 4, loss = 0.02568939
Validation score: 0.993196
Iteration 5, loss = 0.02359463
Validation score: 0.994783
Iteration 6, loss = 0.02064389
Validation score: 0.993649
Iteration 7, loss = 0.01980911
Validation score: 0.993990
Iteration 8, loss = 0.01972849
Validation score: 0.993763
Iteration 9, loss = 0.01797933
Validation score: 0.994670
Iteration 10, loss = 0.01698797
Validation score: 0.994103
MLPClassifier Accuracy:  0.9936759102455546
Current memory usage is 1.100772MB; Peak was 76.383921MB
Elapsed Time: 50.82 seconds


using tenserflow

In [12]:
!pip install tensorflow

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
import tracemalloc

# Standardize the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

model = Sequential()
model.add(Dense(150, input_dim=x_train_scaled.shape[1], activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

tracemalloc.start()

start_time = time.time()

model.fit(x_train_scaled, y_train, epochs=10, batch_size=64, verbose=1, validation_split=0.1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

y_pred_mlp = (model.predict(x_test_scaled) > 0.5).astype(int).flatten()

accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print("MLPClassifier Accuracy: ", accuracy_mlp)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

end_time = time.time()
elapsed_time = end_time - start_time

tracemalloc.stop()

print(f"Elapsed Time: {elapsed_time:.2f} seconds")


Epoch 1/10
1241/1241 [==============================] - 14s 9ms/step - loss: -245373104.0000 - accuracy: 2.3941e-04 - val_loss: -1220203776.0000 - val_accuracy: 2.2681e-04
Epoch 2/10
1241/1241 [==============================] - 13s 11ms/step - loss: -6307403264.0000 - accuracy: 2.3941e-04 - val_loss: -15039948800.0000 - val_accuracy: 2.2681e-04
Epoch 3/10
1241/1241 [==============================] - 10s 8ms/step - loss: -33393297408.0000 - accuracy: 2.3941e-04 - val_loss: -58489409536.0000 - val_accuracy: 2.2681e-04
Epoch 4/10
1241/1241 [==============================] - 5s 4ms/step - loss: -98064089088.0000 - accuracy: 2.3941e-04 - val_loss: -147789594624.0000 - val_accuracy: 2.2681e-04
Epoch 5/10
1241/1241 [==============================] - 5s 4ms/step - loss: -216477843456.0000 - accuracy: 2.3941e-04 - val_loss: -299480514560.0000 - val_accuracy: 2.2681e-04
Epoch 6/10
1241/1241 [==============================] - 11s 9ms/step - loss: -405683077120.0000 - accuracy: 2.3941e-04 - val_lo

knn

In [14]:
!pip install scikit-learn


In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
import tracemalloc

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

knn = KNeighborsClassifier(n_neighbors=5)

tracemalloc.start()

start_time = time.time()

knn.fit(x_train_scaled, y_train)

y_pred_knn = knn.predict(x_test_scaled)

accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("KNeighborsClassifier Accuracy: ", accuracy_knn)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

end_time = time.time()
elapsed_time = end_time - start_time

tracemalloc.stop()

print(f"Elapsed Time: {elapsed_time:.2f} seconds")


KNeighborsClassifier Accuracy:  0.9953164690939882
Current memory usage is 30.273174MB; Peak was 46.397128MB
Elapsed Time: 52.02 seconds


naive bayes


In [16]:
!pip install scikit-learn


In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
import tracemalloc

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

nb_classifier = GaussianNB()

tracemalloc.start()

start_time = time.time()

nb_classifier.fit(x_train_scaled, y_train)

y_pred_nb = nb_classifier.predict(x_test_scaled)

accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Naive Bayes Classifier Accuracy: ", accuracy_nb)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

end_time = time.time()
elapsed_time = end_time - start_time

tracemalloc.stop()

print(f"Elapsed Time: {elapsed_time:.2f} seconds")


Naive Bayes Classifier Accuracy:  0.49145321761219307
Current memory usage is 0.353596MB; Peak was 31.562062MB
Elapsed Time: 0.76 seconds


ivy

In [18]:
!pip install ivy

In [19]:
import ivy
import torch

In [20]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
import tracemalloc
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [21]:
!pip install scikit-learn
!pip install xgboost


without voting classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import tracemalloc
import time

def model_gb(x_train, x_test, y_train, y_test):
    start_time = time.time()
    tracemalloc.start()

    model = GradientBoostingClassifier(random_state=1)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    print("Gradient Boosting Classifier accuracy: ", accuracy_score(y_test, y_pred))

    current = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Current memory usage is {current[0] / 10**6}MB;")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")

def model_knn(x_train, x_test, y_train, y_test):
    start_time = time.time()
    tracemalloc.start()

    model = KNeighborsClassifier()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    print("K-Nearest Neighbors Classifier accuracy: ", accuracy_score(y_test, y_pred))

    current = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Current memory usage is {current[0] / 10**6}MB;")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")

# Assuming x_train, x_test, y_train, and y_test are already defined
model_gb(x_train, x_test, y_train, y_test)
model_knn(x_train, x_test, y_train, y_test)


using voting classifier

In [ ]:
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import tracemalloc
import time

def model(x_train, x_test, y_train, y_test):
    start_time = time.time()
    tracemalloc.start()

    model1 = GradientBoostingClassifier(random_state=1)
    model2 = SVC(random_state=1, probability=True)
    vote_model = VotingClassifier(estimators=[('gb', model1), ('svm', model2)], voting='hard')

    vote_model.fit(x_train, y_train)
    y_pred = vote_model.predict(x_test)

    print("accuracy: ", accuracy_score(y_test, y_pred))

    current = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Current memory usage is {current[0] / 10**6}MB;")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")

model(x_train, x_test, y_train, y_test)